In [2]:
import mujoco 
import mediapy as media 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy.spatial.transform import Rotation as R 
import pickle 
import os 
import cv2 

In [335]:
xml_path = "./env/key_env_v2.xml" 
model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)
with mujoco.Renderer(model, height, width) as renderer:
  mujoco.mj_forward(model, data)
#   renderer.update_scene(data, "track")
#   media.show_image(renderer.render())

n_frames = 200
height = 720 
width = 960
frames = []

# visualize contact frames and forces, make body transparent
options = mujoco.MjvOption()
mujoco.mjv_defaultOption(options)
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = False 
options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True

# tweak scales of contact visualization elements
model.vis.scale.contactwidth = 0.02
model.vis.scale.contactheight = 0.02
model.vis.scale.forcewidth = 0.05
model.vis.map.force = 0.3

# random initial rotational velocity:
mujoco.mj_resetData(model, data)
data.qvel[:3] = 0.1*np.random.randn(3)
data.qvel[3:6] = 1.0*np.random.randn(3)

# Simulate and display video.
with mujoco.Renderer(model, height, width) as renderer:
  for i in range(n_frames):
    while data.time < i/120.0: #1/4x real time
      mujoco.mj_step(model, data)
    renderer.update_scene(data, "track", options)
    frame = renderer.render()
    frames.append(frame)

media.show_video(frames, fps=30)

This browser does not support the video tag.


In [14]:
# TODO: collect sample runs 
# TODO: process sample runs and classify contacts 
# TODO: set up data collection 

xml_path = "./env/key_env_v2.xml" 
model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)
num_trials = 1 
dir_results = "./results/data_v2" 
os.makedirs(dir_results + "/pkl", exist_ok=True)
os.makedirs(dir_results + "/vid", exist_ok=True)
flag_show_video = False 

n_frames = 500  
height = 720 
width = 960
frames = []

# visualize contact frames and forces, make body transparent
options = mujoco.MjvOption()
mujoco.mjv_defaultOption(options)
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = False 
options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True

# tweak scales of contact visualization elements
model.vis.scale.contactwidth = 0.02
model.vis.scale.contactheight = 0.02
model.vis.scale.forcewidth = 0.05
model.vis.map.force = 0.3

for idx_trial in range(num_trials): 

  # define initial conditions 
  x0 = 2*(np.random.rand() - 0.5) * 1e-3 * 5.0
  y0 = 2*(np.random.rand() - 0.5) * 1e-3 * 5.0
  z0 = 2*(np.random.rand() - 0.5) * 1e-3 * 1.0 
  a0 = 2*(np.random.rand() - 0.5) * (np.pi/180) * 3.0 
  b0 = 2*(np.random.rand() - 0.5) * (np.pi/180) * 3.0 
  c0 = 2*(np.random.rand() - 0.5) * (np.pi/180) * 3.0 

  mujoco.mj_resetData(model, data)
  data.qpos = np.array([x0, y0, z0, a0, b0, c0]) 
  data.qvel = np.zeros(6) 
  mujoco.mj_forward(model, data)

  # initialize data structures 
  state_hist = np.zeros((n_frames,1+3+4))   
  contact_hist = [] 
  contact_num = [] 
  contact_geom1 = [] 
  contact_geom2 = [] 
  contact_dist = [] 
  contact_pos = [] 
  contact_frame = [] 
  ctrl_hist = np.zeros((n_frames,1+6)) 
  sensor_hist = np.zeros((n_frames,13))

  # define controller parameters  
  qpos_insert = np.array([0, 0, -10e-3, 0, 0, 0]) 
  z_step = 1e-3 

  # Initialize video writer
  video_path = dir_results + f"/vid/trial_{idx_trial}.mp4"
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
  video_writer = cv2.VideoWriter(video_path, fourcc, 30, (width, height))

  # Simulate and display video.
  with mujoco.Renderer(model, height, width) as renderer:
    for i in range(n_frames): 
      while data.time < i/(30.0*4): #1/4x real time
        mujoco.mj_step(model, data)
      renderer.update_scene(data, "track", options)
      frame = renderer.render()
      frames.append(frame)

      # Convert frame to BGR format for OpenCV
      frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
      video_writer.write(frame_bgr)

      # save data 
      state_hist[i,:] = np.concatenate([np.array([data.time]), data.xpos[2], data.xquat[2]]) 
      contact_num.append(len(data.contact.geom1)) 
      contact_geom1.append(np.array(data.contact.geom1)) 
      contact_geom2.append(np.array(data.contact.geom2)) 
      contact_dist.append(np.array(data.contact.dist))  
      contact_pos.append(np.array(data.contact.pos)) 
      contact_frame.append(np.array(data.contact.frame)) 
      ctrl_hist[i,:] = np.concatenate([np.array([data.time]), data.ctrl])  
      sensor_hist[i,:] = data.sensordata  

      # controller update 
      data.ctrl = data.qpos 
      peg_z_axis = data.xmat[2].reshape(3,3)[:,2]  
      if i > 10: 
        data.ctrl = data.qpos - np.concatenate([peg_z_axis*z_step, np.zeros(3)]) # push in tool -z direction 

  if flag_show_video: 
    media.show_video(frames, fps=30)

  # Release video writer
  video_writer.release() 

  df_state = pd.DataFrame(state_hist, columns=['t','x','y','z','qw','qx','qy','qz']) 

  print(f"Trial {idx_trial} complete.\n") 

  data_dict = {
    'state_hist': state_hist, 
    'contact_num': contact_num,   
    'contact_geom1': contact_geom1,
    'contact_geom2': contact_geom2,
    'contact_dist': contact_dist,
    'contact_pos': contact_pos,
    'contact_frame': contact_frame,
    'ctrl_hist': ctrl_hist
  }

  # save data as pkl file 
  with open(dir_results + f"/pkl/trial_{idx_trial}.pkl", 'wb') as f: 
    pickle.dump(data_dict, f) 
  


Trial 0 complete.



In [16]:
sensor_hist.shape 

(500, 13)